## Distribution Setup

In [ ]:
# from datasets import load_dataset
# from collections import defaultdict
# import random

# # Terapkan random seed untuk hasil yang dapat direproduksi
# random.seed(42)

# # Muat dataset
# dataset = load_dataset("bltlab/queryner")
# label_list = dataset['train'].features['ner_tags'].feature.names

# # --- 1. Konsolidasi Label BIO ---
# # Fungsi untuk menggabungkan label B- dan I- menjadi satu entitas
# consolidated_labels_map = {}
# consolidated_label_names = []
# for label in label_list:
#     if label.startswith('B-'):
#         entity_name = label[2:]
#         consolidated_labels_map[label] = entity_name
#         consolidated_label_names.append(entity_name)
#     elif label.startswith('I-'):
#         entity_name = label[2:]
#         consolidated_labels_map[label] = entity_name
#         consolidated_label_names.append(entity_name)
#     else: # 'O' label
#         consolidated_labels_map[label] = label
#         consolidated_label_names.append(label)
# consolidated_label_names = sorted(list(set(consolidated_label_names)))

# # Daftar label yang terpengaruh
# labels_to_plot = ['condition', 'quantity', 'price', 'origin', 'time', 'product_number']

# # Fungsi untuk menghitung entitas di split dan mengkonsolidasikan
# def count_and_consolidate(split_data):
#     counts = defaultdict(int)
#     for example in split_data:
#         for tag_id in example['ner_tags']:
#             original_label = label_list[tag_id]
#             consolidated_label = consolidated_labels_map[original_label]
#             counts[consolidated_label] += 1
#     return counts

# # Fungsi untuk menghitung data dan persentase
# def calculate_distribution(counts, labels_to_check):
#     plot_data = defaultdict(dict)
#     total_counts = {}

#     for label in labels_to_check:
#         total_counts[label] = sum(counts[split][label] for split in ['train', 'validation', 'test'])

#     for label in labels_to_check:
#         total = total_counts[label]
#         if total > 0:
#             for split in ['train', 'validation', 'test']:
#                 count = counts[split][label]
#                 percentage = (count / total) * 100 if total > 0 else 0
#                 plot_data[label][split] = {'percentage': round(percentage, 2), 'count': count}
#         else:
#             for split in ['train', 'validation', 'test']:
#                 plot_data[label][split] = {'percentage': 0, 'count': 0}

#     return plot_data

# # --- Tahap Awal: Hitung Distribusi Sebelum Penyesuaian ---
# print("Menghitung distribusi entitas sebelum penyesuaian...")
# initial_counts = {
#     'train': count_and_consolidate(dataset['train']),
#     'validation': count_and_consolidate(dataset['validation']),
#     'test': count_and_consolidate(dataset['test'])
# }
# initial_distribution = calculate_distribution(initial_counts, labels_to_plot)

# # --- Tahap Penyesuaian Distribusi Entitas ---
# print("\nMelakukan penyesuaian distribusi entitas...")
# rebalanced_dataset = {split: list(dataset[split]) for split in dataset.keys()}

# def move_samples(source_split_name, dest_split_name, label_to_move, num_to_move):
#     global rebalanced_dataset
    
#     source_split = rebalanced_dataset[source_split_name]
#     dest_split = rebalanced_dataset[dest_split_name]
    
#     potential_indices = [i for i, example in enumerate(source_split) if any(label_to_move in label_list[tag_id] for tag_id in example['ner_tags'])]
    
#     if len(potential_indices) >= num_to_move:
#         indices_to_remove = random.sample(potential_indices, num_to_move)
#     else:
#         indices_to_remove = potential_indices
    
#     if len(indices_to_remove) > 0:
#         samples_to_move = [source_split[i] for i in indices_to_remove]
#         for i in sorted(indices_to_remove, reverse=True):
#             del source_split[i]
#         dest_split.extend(samples_to_move)

# # Terapkan penyesuaian yang dijelaskan:
# move_samples('train', 'test', 'condition', 15)
# move_samples('test', 'train', 'quantity', 5)
# move_samples('train', 'validation', 'price', 3)
# move_samples('train', 'test', 'price', 2)
# move_samples('test', 'validation', 'origin', 3)
# move_samples('train', 'test', 'time', 3)
# move_samples('test', 'validation', 'product_number', 3)

# # --- Tahap Akhir: Hitung ulang distribusi setelah penyesuaian ---
# print("\nPenyesuaian selesai. Menghitung ulang distribusi entitas...")
# final_counts = {
#     'train': count_and_consolidate(rebalanced_dataset['train']),
#     'validation': count_and_consolidate(rebalanced_dataset['validation']),
#     'test': count_and_consolidate(rebalanced_dataset['test'])
# }
# final_distribution = calculate_distribution(final_counts, labels_to_plot)

In [ ]:
# import os
# import json

# # Create directories if they don't exist
# os.makedirs('data/raw', exist_ok=True)
# os.makedirs('data/processed', exist_ok=True)
# os.makedirs('data/distribution', exist_ok=True)

# # Save original dataset to raw folder
# print("\nSaving original dataset...")
# for split in dataset.keys():
#     dataset[split].to_json(f'data/raw/{split}.json')
# print("Original dataset saved to data/raw/")

# # Save rebalanced dataset to processed folder
# print("\nSaving processed dataset...")
# for split, data in rebalanced_dataset.items():
#     # Convert list of examples to a format that can be saved
#     processed_data = {
#         'examples': data
#     }
#     with open(f'data/processed/{split}.json', 'w', encoding='utf-8') as f:
#         json.dump(processed_data, f, ensure_ascii=False, indent=2)
# print("Processed dataset saved to data/processed/")

# # Save distributions to distribution folder
# print("\nSaving distribution data...")
# distribution_data = {
#     'initial_distribution': initial_distribution,
#     'final_distribution': final_distribution
# }
# with open('data/distribution/label_distributions.json', 'w', encoding='utf-8') as f:
#     json.dump(distribution_data, f, ensure_ascii=False, indent=2)
# print("Distribution data saved to data/distribution/label_distributions.json")

In [ ]:
# import json
# from pathlib import Path

# def reformat_jsonl_file(file_path):
#     examples = []

#     with open(file_path, 'r', encoding='utf-8') as f:
#         for line in f:
#             if line.strip():  # Skip empty lines
#                 data = json.loads(line)
#                 examples.append(data)
    
#     # Create output structure
#     output_data = {"examples": examples}
    
#     # Overwrite the original file
#     with open(file_path, 'w', encoding='utf-8') as f:
#         json.dump(output_data, f, indent=2, ensure_ascii=False)
    
#     print(f"✓ Reformatted {file_path}")
#     print(f"  Total examples: {len(examples)}")


# # Usage examples:
# if __name__ == "__main__":
#     # Single file
#     reformat_jsonl_file(r"D:\Dafa\Project\queryner-kd\data\raw\train.json")
#     reformat_jsonl_file(r"D:\Dafa\Project\queryner-kd\data\raw\validation.json")
#     reformat_jsonl_file(r"D:\Dafa\Project\queryner-kd\data\raw\test.json")
    
#     # Or batch process all files
#     # raw_dir = Path(r"D:\Dafa\Project\queryner-kd\data\raw")
#     # for json_file in raw_dir.glob("*.json"):
#     #     reformat_jsonl_file(json_file)

✓ Reformatted D:\Dafa\Project\queryner-kd\data\raw\train.json
  Total examples: 7841
✓ Reformatted D:\Dafa\Project\queryner-kd\data\raw\validation.json
  Total examples: 871
✓ Reformatted D:\Dafa\Project\queryner-kd\data\raw\test.json
  Total examples: 993


In [ ]:
# with open(r"D:\Dafa\Project\queryner-kd\data\processed\train.json", 'r', encoding='utf-8') as f:
#     print(len(json.load(f)['examples']))
# with open(r"D:\Dafa\Project\queryner-kd\data\processed\validation.json", 'r', encoding='utf-8') as f:
#     print(len(json.load(f)['examples']))
# with open(r"D:\Dafa\Project\queryner-kd\data\processed\test.json", 'r', encoding='utf-8') as f:
#     print(len(json.load(f)['examples']))

In [ ]:
import json
from pathlib import Path

def compare_datasets(raw_dir, processed_dir):
    """
    Compare raw and processed datasets and find differences.
    
    Args:
        raw_dir: Path to raw data directory
        processed_dir: Path to processed data directory
    """
    
    splits = ["train", "validation", "test"]
    
    for split in splits:
        print(f"\n{'='*80}")
        print(f"COMPARING {split.upper()}")
        print(f"{'='*80}")
        
        # Load raw data
        raw_path = Path(raw_dir) / f"{split}.json"
        with open(raw_path, 'r', encoding='utf-8') as f:
            raw_data = json.load(f)['examples']
        
        # Load processed data
        processed_path = Path(processed_dir) / f"{split}.json"
        with open(processed_path, 'r', encoding='utf-8') as f:
            processed_data = json.load(f)['examples']
        
        print(f"Raw: {len(raw_data)} examples")
        print(f"Processed: {len(processed_data)} examples")
        print(f"Difference: {len(raw_data) - len(processed_data)} examples\n")
        
        # Find differences
        raw_set = set()
        processed_set = set()
        
        # Convert to hashable format (tuple of tokens and tags)
        for example in raw_data:
            key = (tuple(example['tokens']), tuple(example['ner_tags']))
            raw_set.add(key)
        
        for example in processed_data:
            key = (tuple(example['tokens']), tuple(example['ner_tags']))
            processed_set.add(key)
        
        # Find differences
        only_in_raw = raw_set - processed_set
        only_in_processed = processed_set - raw_set
        
        print(f"Only in RAW (removed): {len(only_in_raw)} examples")
        print(f"Only in PROCESSED (added): {len(only_in_processed)} examples")
        
        # Display first few differences
        if only_in_raw:
            print(f"\nFirst 5 examples REMOVED from raw -> processed:")
            for i, (tokens, tags) in enumerate(list(only_in_raw)[:5], 1):
                print(f"\n{i}. Tokens: {tokens}")
                print(f"   Tags: {tags}")
        
        if only_in_processed:
            print(f"\nFirst 5 examples ADDED in processed:")
            for i, (tokens, tags) in enumerate(list(only_in_processed)[:5], 1):
                print(f"\n{i}. Tokens: {tokens}")
                print(f"   Tags: {tags}")
        
        # Save detailed diff report
        save_diff_report(split, only_in_raw, only_in_processed)


def save_diff_report(split, only_in_raw, only_in_processed):
    """Save detailed diff report to file"""
    
    output_dir = Path("results/data_analysis")
    output_dir.mkdir(parents=True, exist_ok=True)
    
    report_path = output_dir / f"diff_{split}.json"
    
    report = {
        "split": split,
        "only_in_raw": [
            {
                "tokens": list(tokens),
                "ner_tags": list(tags)
            }
            for tokens, tags in only_in_raw
        ],
        "only_in_processed": [
            {
                "tokens": list(tokens),
                "ner_tags": list(tags)
            }
            for tokens, tags in only_in_processed
        ]
    }
    
    with open(report_path, 'w', encoding='utf-8') as f:
        json.dump(report, f, indent=2, ensure_ascii=False)
    
    print(f"\n✓ Detailed report saved to: {report_path}")


# Usage
if __name__ == "__main__":
    raw_dir = r"D:\Dafa\Project\queryner-kd\data\raw"
    processed_dir = r"D:\Dafa\Project\queryner-kd\data\processed"
    
    compare_datasets(raw_dir, processed_dir)


COMPARING TRAIN
Raw: 7841 examples
Processed: 7823 examples
Difference: 18 examples

Only in RAW (removed): 23 examples
Only in PROCESSED (added): 5 examples

First 5 examples REMOVED from raw -> processed:

1. Tokens: ('rustic', 'grey', 'wall', 'mirror')
   Tags: (5, 3, 9, 10)

2. Tokens: ('director', 'chair', 'without', 'arms')
   Tags: (17, 9, 5, 6)

3. Tokens: ('good', 'drawing', 'tablets', 'with', 'screens')
   Tags: (5, 9, 10, 17, 18)

4. Tokens: ('10m', 'waterproof', 'led', 'strip', 'lights', 'outdoor', 'without', 'controller')
   Tags: (17, 18, 9, 10, 10, 17, 5, 6)

5. Tokens: ('03', 'dodge', 'ram', 'visor')
   Tags: (33, 11, 9, 10)

First 5 examples ADDED in processed:

1. Tokens: ('bulk', 'tote', 'bags')
   Tags: (29, 9, 10)

2. Tokens: ('2', 'piece', 'swimsuits', 'for', 'women')
   Tags: (29, 30, 9, 17, 13)

3. Tokens: ('1', 'huba', 'buba', 'origanal', 'not', 'exspencive')
   Tags: (29, 25, 26, 5, 23, 24)

4. Tokens: ('dog', 'food', '100', 'lb', 'bag')
   Tags: (9, 10, 29, 

## Compare LR, Data, and CRF in BERT baseline (12 exps)

In [5]:
import json
import pandas as pd
from pathlib import Path

# Load the baseline results
summary_path = Path(r"D:\Dafa\Project\queryner-kd\results\baseline\1-12-bert\summary-1-12-bert-token-mean.json")
with open(summary_path, 'r') as f:
    results = json.load(f)

# Parse all experiments
experiments = results['successful_experiments']

# Extract data
data = []
for exp in experiments:
    # Parse experiment name: format is "teacher_{data_type}_{lr}_{crf_type}"
    parts = exp['exp_name'].split('_')
    data_type = parts[1]  # 'processed' or 'raw'
    lr = parts[2]  # '1e-5', '2e-5', '5e-5'
    crf_type = parts[3]  # 'crf' or 'nocrf'
    
    data.append({
        'data_type': data_type,
        'lr': lr,
        'crf_type': crf_type,
        'best_val_f1': exp['best_val_f1'],
        'test_f1': exp['test_f1']
    })

df = pd.DataFrame(data)

print("=" * 80)
print("TABLE 1: LEARNING RATE COMPARISON (Average across CRF and Data types)")
print("=" * 80)

# Group by learning rate
lr_comparison = df.groupby('lr')[['best_val_f1', 'test_f1']].agg(['mean', 'std', 'count'])
lr_summary = pd.DataFrame({
    'LR Value': lr_comparison.index,
    'Avg Best Val F1': lr_comparison['best_val_f1']['mean'].values,
    'Std Best Val F1': lr_comparison['best_val_f1']['std'].values,
    'Avg Test F1': lr_comparison['test_f1']['mean'].values,
    'Std Test F1': lr_comparison['test_f1']['std'].values,
    'Count': lr_comparison['best_val_f1']['count'].values
}).set_index('LR Value')

print(lr_summary.to_string())
print()

print("=" * 80)
print("TABLE 2: DATA TYPE COMPARISON (Average across LR and CRF)")
print("=" * 80)

# Group by data type
data_comparison = df.groupby('data_type')[['best_val_f1', 'test_f1']].agg(['mean', 'std', 'count'])
data_summary = pd.DataFrame({
    'Data Type': data_comparison.index,
    'Avg Best Val F1': data_comparison['best_val_f1']['mean'].values,
    'Std Best Val F1': data_comparison['best_val_f1']['std'].values,
    'Avg Test F1': data_comparison['test_f1']['mean'].values,
    'Std Test F1': data_comparison['test_f1']['std'].values,
    'Count': data_comparison['best_val_f1']['count'].values
}).set_index('Data Type')

print(data_summary.to_string())
print()

print("=" * 80)
print("DETAILED BREAKDOWN")
print("=" * 80)

# Show all combinations
print("\nAll experiments:")
print(df.to_string(index=False))
print()

# Summary statistics
print("Overall Statistics:")
print(f"  Mean Best Val F1: {df['best_val_f1'].mean():.4f}")
print(f"  Mean Test F1: {df['test_f1'].mean():.4f}")
print(f"  Best Val F1 Range: [{df['best_val_f1'].min():.4f}, {df['best_val_f1'].max():.4f}]")
print(f"  Test F1 Range: [{df['test_f1'].min():.4f}, {df['test_f1'].max():.4f}]")

TABLE 1: LEARNING RATE COMPARISON (Average across CRF and Data types)
          Avg Best Val F1  Std Best Val F1  Avg Test F1  Std Test F1  Count
LR Value                                                                   
1e-5             0.739247         0.004191     0.659698     0.002501      4
2e-5             0.734300         0.003383     0.660347     0.006639      4
5e-5             0.731520         0.002223     0.662997     0.007341      4

TABLE 2: DATA TYPE COMPARISON (Average across LR and CRF)
           Avg Best Val F1  Std Best Val F1  Avg Test F1  Std Test F1  Count
Data Type                                                                   
processed         0.735036         0.004994     0.662379     0.006556      6
raw               0.735009         0.004465     0.659649     0.004469      6

DETAILED BREAKDOWN

All experiments:
data_type   lr crf_type  best_val_f1  test_f1
processed 1e-5      crf     0.733820 0.658730
processed 1e-5    nocrf     0.743655 0.658456
process